In [ ]:
# --- 1. Imports ---
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
from tqdm import tqdm

# --- 2. Utility Functions ---
def load(fn):
    with open(fn, 'r', encoding='utf-8') as f:
        return json.load(f)

def process_date_features(articles):
    years = np.array([a['year'] for a in articles]).reshape(-1, 1)
    months = np.array([a['month'] for a in articles]).reshape(-1, 1)
    day_of_week = np.array([a['day_of_week'] for a in articles])
    hour = np.array([a['hour'] for a in articles])

    year_scaler = StandardScaler()
    month_scaler = StandardScaler()
    years_scaled = year_scaler.fit_transform(years)
    months_scaled = month_scaler.fit_transform(months)

    day_sin = np.sin(2 * np.pi * day_of_week / 7)
    day_cos = np.cos(2 * np.pi * day_of_week / 7)
    hour_sin = np.sin(2 * np.pi * hour / 24)
    hour_cos = np.cos(2 * np.pi * hour / 24)

    date_feats = np.concatenate([years_scaled, months_scaled, day_sin[:, None], day_cos[:, None], hour_sin[:, None], hour_cos[:, None]], axis=1)
    return date_feats

def extract_topics_from_url(url):
    parts = url.split('/')
    topic = parts[3] if len(parts) > 3 else 'none'
    subtopic = parts[4] if len(parts) > 4 else 'none'
    return topic, subtopic

# --- 3. Dataset ---
class NewsDataset(Dataset):
    def __init__(self, bert_vectors, topic_ids, subtopic_ids, date_feats, targets):
        self.X = torch.tensor(bert_vectors, dtype=torch.float32)
        self.topic_ids = torch.tensor(topic_ids, dtype=torch.long)
        self.subtopic_ids = torch.tensor(subtopic_ids, dtype=torch.long)
        self.date_feats = torch.tensor(date_feats, dtype=torch.float32)
        self.y = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.topic_ids[idx], self.subtopic_ids[idx], self.date_feats[idx], self.y[idx]

# --- 4. Model ---
class MLPWithEmbeddings(nn.Module):
    def __init__(self, input_dim, num_topics, num_subtopics):
        super().__init__()
        self.topic_embedding = nn.Embedding(num_topics, 16)
        self.subtopic_embedding = nn.Embedding(num_subtopics, 24)

        self.model = nn.Sequential(
            nn.Linear(input_dim + 16 + 24 + 6, 1024),
            nn.BatchNorm1d(1024),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 512),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(512, 128),
            nn.GELU(),
            nn.Linear(128, 1),
            nn.Softplus()
        )

    def forward(self, x_text, topic_ids, subtopic_ids, date_feats):
        topic_embed = self.topic_embedding(topic_ids)
        subtopic_embed = self.subtopic_embedding(subtopic_ids)
        x = torch.cat([x_text, topic_embed, subtopic_embed, date_feats], dim=1)
        return self.model(x)

# --- 5. Main Class ---
class RTVSloBERT:
    def __init__(self, batch_size=170, epochs=70, learning_rate=1e-4, l2_lambda=1e-3, eval_split=0.05):
        self.batch_size = batch_size
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.l2_lambda = l2_lambda
        self.eval_split = eval_split

    def fit(self, train_data):
        for a in train_data:
            dt = pd.to_datetime(a['date'])
            a['year'] = dt.year
            a['month'] = dt.month
            a['day_of_week'] = dt.weekday()
            a['hour'] = dt.hour

        raw_targets = [a['n_comments'] for a in train_data]
        targets = [np.log1p(t) for t in raw_targets]
        bert_vectors = torch.load("train/sloberta_embeddings.pt", map_location="cpu").numpy()

        topics = []
        subtopics = []
        for a in train_data:
            topic, subtopic = extract_topics_from_url(a['url'])
            topics.append(topic)
            subtopics.append(subtopic if subtopic != "NO_SUBTOPIC" else "none")

        date_feats = process_date_features(train_data)

        self.topic_encoder = LabelEncoder().fit(topics)
        self.subtopic_encoder = LabelEncoder().fit(subtopics)

        topic_ids = self.topic_encoder.transform(topics)
        subtopic_ids = self.subtopic_encoder.transform(subtopics)

        X_train, X_val, topic_train, topic_val, subtopic_train, subtopic_val, date_train, date_val, y_train, y_val = train_test_split(
            bert_vectors, topic_ids, subtopic_ids, date_feats, targets, test_size=self.eval_split, random_state=42
        )

        train_dataset = NewsDataset(X_train, topic_train, subtopic_train, date_train, y_train)
        val_dataset = NewsDataset(X_val, topic_val, subtopic_val, date_val, y_val)

        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=self.batch_size)

        input_dim = bert_vectors.shape[1]
        self.model = MLPWithEmbeddings(input_dim, len(self.topic_encoder.classes_), len(self.subtopic_encoder.classes_))

        self._train(self.model, train_loader, val_loader)

    def _train(self, model, train_loader, val_loader):
        optimizer = torch.optim.Adam(model.parameters(), lr=self.learning_rate, weight_decay=self.l2_lambda)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
        criterion = nn.HuberLoss(delta=5.0)

        best_val_mae = float('inf')
        best_model_state = None

        for epoch in range(self.epochs):
            model.train()
            train_mae_list = []
            for x_batch, topic_ids, subtopic_ids, date_feats, y_batch in train_loader:
                optimizer.zero_grad()
                y_pred = model(x_batch, topic_ids, subtopic_ids, date_feats).squeeze()
                loss = criterion(y_pred, y_batch)
                loss.backward()
                optimizer.step()

                y_pred_clamped = torch.clamp(y_pred, -10, 10)
                y_batch_clamped = torch.clamp(y_batch, -10, 10)
                mae = torch.mean(torch.abs(torch.expm1(y_pred_clamped) - torch.expm1(y_batch_clamped))).item()
                train_mae_list.append(mae)

            model.eval()
            val_mae_list = []
            with torch.no_grad():
                for x_batch, topic_ids, subtopic_ids, date_feats, y_batch in val_loader:
                    y_pred = model(x_batch, topic_ids, subtopic_ids, date_feats).squeeze()
                    y_pred_clamped = torch.clamp(y_pred, -10, 10)
                    y_batch_clamped = torch.clamp(y_batch, -10, 10)
                    mae = torch.mean(torch.abs(torch.expm1(y_pred_clamped) - torch.expm1(y_batch_clamped))).item()
                    val_mae_list.append(mae)

            train_mae = np.mean(train_mae_list)
            val_mae = np.mean(val_mae_list)
            scheduler.step(val_mae)

            if val_mae < best_val_mae:
                best_val_mae = val_mae
                best_model_state = model.state_dict()

            print(f"Epoch {epoch+1} - Train MAE: {train_mae:.2f} | Val MAE: {val_mae:.2f}")

        if best_model_state is not None:
            model.load_state_dict(best_model_state)

    def predict(self, test_data, bert_vectors):
        for a in test_data:
            dt = pd.to_datetime(a['date'])
            a['year'] = dt.year
            a['month'] = dt.month
            a['day_of_week'] = dt.weekday()
            a['hour'] = dt.hour

        topics = []
        subtopics = []
        for a in test_data:
            topic, subtopic = extract_topics_from_url(a['url'])
            topics.append(topic)
            subtopics.append(subtopic if subtopic != "NO_SUBTOPIC" else "none")

        date_feats = process_date_features(test_data)

        topic_ids = [self.topic_encoder.classes_[0] if t not in self.topic_encoder.classes_ else t for t in topics]
        subtopic_ids = [self.subtopic_encoder.classes_[0] if s not in self.subtopic_encoder.classes_ else s for s in subtopics]

        topic_ids = self.topic_encoder.transform(topic_ids)
        subtopic_ids = self.subtopic_encoder.transform(subtopic_ids)

        X = torch.tensor(bert_vectors, dtype=torch.float32)
        topic_ids = torch.tensor(topic_ids, dtype=torch.long)
        subtopic_ids = torch.tensor(subtopic_ids, dtype=torch.long)
        date_feats = torch.tensor(date_feats, dtype=torch.float32)

        self.model.eval()
        with torch.no_grad():
            preds = self.model(X, topic_ids, subtopic_ids, date_feats).squeeze().numpy()
            return np.clip(np.expm1(preds), 0, None)

# --- 6. Ensemble Training ---
if __name__ == '__main__':
    train = load("../data/rtvslo_train.json")
    test = load("../data/rtvslo_test.json")

    bert_vectors_test = torch.load("test_final/sloberta_embeddings_final.pt", map_location="cpu").numpy()

    all_predictions = []

    for run in range(70):
        print(f"🔵 Training model {run+1}/70...")
        model = RTVSloBERT(
            eval_split=0.05,
            batch_size=170,
            epochs=70,
            learning_rate=1e-4,
            l2_lambda=1e-3
        )
        model.fit(train)
        preds = model.predict(test, bert_vectors_test)
        all_predictions.append(preds)

    print("🔵 Averaging all predictions...")
    avg_preds = np.mean(np.stack(all_predictions), axis=0)
    np.savetxt("ensemble_predictions.txt", avg_preds, fmt="%.4f")
    print("✅ Saved final averaged predictions!")


/tmp/ipykernel_38801/2826453792.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  bert_vectors = torch.load("train/sloberta_embeddings.pt", map_location="cpu").numpy()


Epoch 1 - Train MAE: 30.96 | Val MAE: 27.81
Epoch 2 - Train MAE: 28.03 | Val MAE: 26.51
Epoch 3 - Train MAE: 27.24 | Val MAE: 26.29
Epoch 4 - Train MAE: 26.65 | Val MAE: 26.04
Epoch 5 - Train MAE: 26.16 | Val MAE: 25.43
Epoch 6 - Train MAE: 25.78 | Val MAE: 25.35
Epoch 7 - Train MAE: 25.56 | Val MAE: 25.95
Epoch 8 - Train MAE: 25.15 | Val MAE: 24.94
Epoch 9 - Train MAE: 24.83 | Val MAE: 24.25
Epoch 10 - Train MAE: 24.49 | Val MAE: 24.62
Epoch 11 - Train MAE: 24.28 | Val MAE: 24.85
Epoch 12 - Train MAE: 23.90 | Val MAE: 24.39
Epoch 13 - Train MAE: 23.75 | Val MAE: 24.22
Epoch 14 - Train MAE: 23.50 | Val MAE: 24.23
Epoch 15 - Train MAE: 23.18 | Val MAE: 24.42
Epoch 16 - Train MAE: 22.97 | Val MAE: 23.88
Epoch 17 - Train MAE: 22.71 | Val MAE: 24.01
Epoch 18 - Train MAE: 22.49 | Val MAE: 24.73
Epoch 19 - Train MAE: 22.09 | Val MAE: 23.78
Epoch 20 - Train MAE: 21.94 | Val MAE: 24.09
Epoch 21 - Train MAE: 21.61 | Val MAE: 25.15
Epoch 22 - Train MAE: 21.27 | Val MAE: 24.29
Epoch 23 - Train MA

/tmp/ipykernel_38801/2826453792.py:203: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  bert_vectors_validation = torch.load("validation/sloberta_embeddings_val.pt", map_locat

In [2]:
import numpy as np
import json
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

# --- 1. Load ---
preds = np.loadtxt("predictions_val.txt")

# Optional: If you have the true labels available (for evaluation)
# e.g., validation set where you know true n_comments
# with open("../data/rtvslo_validation.json", "r", encoding="utf-8") as f:
#     val_articles = json.load(f)
# y_true = np.array([a['n_comments'] for a in val_articles], dtype=np.float32)
# have_true_labels = True

# --- 2. Postprocessing: Quantile Smoothing ---

# Step 1: Clip extreme values
max_allowed = 3000
preds = np.clip(preds, 0, max_allowed)

# Step 2: Quantile smoothing
q_low = np.percentile(preds, 1)
q_high = np.percentile(preds, 99)

# Linearly squash very small and very large predictions
def quantile_smooth(x):
    if x < q_low:
        return x * 0.7  # shrink low values
    elif x > q_high:
        return q_high # shrink heavy tails
    else:
        return x

vectorized_smooth = np.vectorize(quantile_smooth)
smoothed_preds = vectorized_smooth(preds)

# --- 3. Save ---
np.savetxt("adj_predictions.txt", smoothed_preds, fmt="%.4f")

print("✅ Saved adj_predictions.txt with quantile smoothing.")

✅ Saved adj_predictions.txt with quantile smoothing.


In [5]:
import numpy as np
import json
from sklearn.metrics import mean_absolute_error

# 1. Load your predictions
preds = np.loadtxt("adj_predictions.txt")

# 2. Load your true y-values from dataset_val.json
with open("../data/rtvslo_validation.json", "r", encoding="utf-8") as f:
    val_articles = json.load(f)

# 3. Extract true n_comments
y_true = np.array([a["n_comments"] for a in val_articles], dtype=np.float32)

# 4. Check lengths
assert len(preds) == len(y_true), f"Length mismatch: preds={len(preds)}, y_true={len(y_true)}"

# 5. Calculate MAE
mae = mean_absolute_error(y_true, preds)

print(f"📊 MAE between predictions and true values: {mae:.2f}")


📊 MAE between predictions and true values: 30.16
